# Test Affinity and comp algorithm on liquid
- this algorithm is usually fast and accurate
- but it can fail to converge, causing major problems


In [ ]:
import numpy as np
from os import path
import pandas as pd
import scipy.optimize as opt
from scipy import optimize
import scipy.linalg as lin
import scipy as sp
import sys
import sympy as sym

from collections import OrderedDict as odict

import matplotlib.pyplot as plt

import sunkenhull as hull
from thermoengine.model import GeoCompDB, SysComp

Required ENKI modules (ignore the error message from Rubicon running under Python 3.6+)

In [ ]:
from thermoengine import coder, core, phases, model, equilibrate

In [ ]:
modelDB = model.Database(database='Berman')

## Assume MORB Bulk Composition


In [ ]:
compDB = GeoCompDB()

In [ ]:
sysID = 'MORB_primitive'
CO2='none'
# H2O='none'
H2O = 'input'
syscomp = compDB.get_syscomp(
    sysID, components='oxides',CO2=CO2,H2O=H2O)
display(syscomp.wt_comp())
display(syscomp.mol_comp('oxides'))

In [ ]:
elem_comp = syscomp.mol_comp(components='elems')
sys_elems=elem_comp.columns.values
bulk_comp = elem_comp.values[0]

## P/T conditions selected to involve many phases present

In [ ]:
T = 1175+273.15
T= 1600+273
P = 1e3

## Initialize assemblage w/ pure liquid
- if omnicomponent phase is present this will produce good initial guess for chempot
- assume pure liquid for MORB composition system


In [ ]:
mol_oxides=syscomp.mol_comp(components='oxides')
mol_oxides['CO2']=0
mol_oxides.values.squeeze()

In [ ]:
liq = modelDB.get_phase('Liq')

In [ ]:
mol_endmem = liq.calc_endmember_comp(mol_oxides.values.squeeze(),method='intrinsic')
mol_endmem/=mol_endmem.sum()

In [ ]:
chempot = liq.chem_potential(T, P,  mol=mol_endmem).squeeze()

In [ ]:
dG = liq.gibbs_energy(T, P, mol=mol_endmem)-chempot.dot(mol_endmem)
dG

In [ ]:
# lin.lstsq()

In [ ]:
liq_elem_comp = pd.DataFrame(liq.props['element_comp'],index=liq.endmember_names, 
                             columns=SysComp.PERIODIC_ORDER)[sys_elems]
liq_elem_comp


## verify that liquid composition calculated correctly

In [ ]:
elem_diff = liq_elem_comp.T.dot(mol_endmem)-elem_comp
elem_diff.abs()<1e-10

In [ ]:
chempot_elems = lin.lstsq(liq_elem_comp, chempot)[0]
chempot_elems

In [ ]:
mu = chempot.copy()
mu[mol_endmem==0]=0
mu

In [ ]:
noise = .3
# noise=0

In [ ]:
X_init = mol_endmem*np.exp(noise*np.random.randn(mu.size))
X_init[X_init<0] = 0
X_init[X_init>1] = 1
X_init[mol_endmem==0] = 0

In [ ]:
# X_init /= X_init.sum()

In [ ]:
liq.chem_potential(T, P,  mol=X_init).squeeze()

## Cold-start legacy version

In [ ]:
A0, X0 = liq.affinity_and_comp_legacy(T, P, mu)
print(A0)
logdx = np.log(X0/mol_endmem)
logdx[mol_endmem==0] = 0
logdx

In [ ]:
A, X = liq.affinity_and_comp(T, P, mu)
print(A)
logdx = np.log(X/mol_endmem)
logdx[mol_endmem==0] = 0
logdx

In [ ]:
dmu = (liq.chem_potential(T, P, mol=X)-mu).squeeze()
dmu[mu==0]=0
np.dot(dmu, X)

In [ ]:
site_m = liq.exchange_equil._est_site_mult(T, P)

## Cold-start converges rapidly

In [ ]:
A, X = liq.affinity_and_comp(T, P, mu, debug=True, converge_method='lstsq', site_m=site_m)

In [ ]:
mol_lims = [1e-4,1]
plt.figure()
plt.loglog(mol_endmem, X, 'bo')
plt.loglog(mol_endmem, X_init, 'rx')
plt.plot(mol_lims, mol_lims, 'r--')
# plt.xlim(0,1)
# plt.ylim(0,1)
plt.xlim(mol_lims)
plt.ylim(mol_lims)

## Warm-start also converges
- roughly same convergence time for cold start and warm start

In [ ]:
A, X = liq.affinity_and_comp(T, P, mu, X_init=X_init, debug=True, converge_method='approx', site_m=site_m)

In [ ]:
display(A)
display(X)

In [ ]:
dmu = (liq.chem_potential(T, P, mol=X)-mu).squeeze()
dmu[mu==0]=0
np.dot(dmu, X)

In [ ]:

np.dot(liq.chem_potential(T, P, mol=X)-mu, X)

In [ ]:
np.log(X/mol_endmem)[mol_endmem!=0]

In [ ]:
Xdiff = X-mol_endmem
Xdiff_max = np.max(np.abs(Xdiff))
print('max Xdiff = ', Xdiff_max)

In [ ]:
plt.figure()
plt.plot(mol_endmem, X, 'o')
plt.plot([0,1],[0,1], 'r--')
plt.xlim(0,1)
plt.ylim(0,1)

In [ ]:
mol_lims = [1e-4,1]
plt.figure()
plt.loglog(mol_endmem, X, 'bo')
plt.loglog(mol_endmem, X_init, 'rx')
plt.plot(mol_lims, mol_lims, 'r--')
# plt.xlim(0,1)
# plt.ylim(0,1)
plt.xlim(mol_lims)
plt.ylim(mol_lims)

In [ ]:
inds = np.argsort(mol_endmem)[::-1]

In [ ]:

mol_pure = np.eye(mol_endmem.size)
mol_pure[0]

In [ ]:
'''
A, X = liq.affinity_and_comp(T, P, mu, X_init=mol_pure[inds[6]], ATOL=1e-3, XTOL=1e-4, iter_max=50, debug=True)
plt.figure()
plt.plot(mol_endmem, X, 'o')
plt.xlim(0,1)
plt.ylim(0,1)
np.max(X-mol_endmem)
'''